In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
#from transformers import Trainer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import math
import random
import re
import matplotlib.pyplot as plt
import time
import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchmetrics import F1Score

# import datasets
# from datasets import Dataset, load_metric

import warnings
warnings.filterwarnings('ignore')

# Data processing

In [ ]:
dt = {'album_favorites': 'Int64', 'album_title': 'category', 
 'artist_favorites': 'Int64', 'artist_name': 'category', 'track_favorites':'Int64', 'track_genre_top': 'category',
 'track_genres': 'object', 'track_genres_all':'object', 'track_interest':'Int64', 'track_listens': 'Int64',
 'track_title': 'object' }
csv_path = '/content/drive/My Drive/cs267a/data.csv'
all_data = pd.read_csv(csv_path, dtype= dt)

print(all_data.dtypes)


album_favorites        Int64
album_title         category
artist_favorites       Int64
artist_name         category
track_favorites        Int64
track_genre_top     category
track_genres          object
track_genres_all      object
track_interest         Int64
track_listens          Int64
track_title           object
dtype: object


In [ ]:
# all_data = all_data.drop(all_data[all_data['track_title'].isna()].index)
# all_data = all_data.drop(all_data[all_data['artist_name'].isna()].index)
all_data = all_data.drop(all_data[all_data['album_title'].isna()].index)
all_data = all_data.drop(all_data[all_data['artist_favorites'] == -1].index)
all_data = all_data.drop(all_data[all_data['album_favorites'] == -1].index)
print(f"dataset size: {len(all_data)}")

dataset size: 9783


In [ ]:
#data_frame = all_data[:2000]
data_frame = all_data
print(f"dataset size: {len(data_frame)}")

dataset size: 9783


In [ ]:
def get_level(q75, q50, q25, val): 
    if val > q75: return 3
    if val > q50: return 2
    if val > q25: return 1
    return 0

def convert_to_level(data_frame):
  #split the numerical value into 4 levels.
  album_fav_q75 = data_frame['album_favorites'].quantile(0.75)
  album_fav_q50 = data_frame['album_favorites'].quantile(0.50)
  album_fav_q25 = data_frame['album_favorites'].quantile(0.25)
  data_frame['album_favorites_level'] = data_frame.apply(lambda row: get_level(album_fav_q75,album_fav_q50,album_fav_q25, row['album_favorites']), axis = 1)

  artist_fav_q75 = data_frame['artist_favorites'].quantile(0.75)
  artist_fav_q50 = data_frame['artist_favorites'].quantile(0.50)
  artist_fav_q25 = data_frame['artist_favorites'].quantile(0.25)
  data_frame['artist_favorites_level'] = data_frame.apply(lambda row: get_level(artist_fav_q75,artist_fav_q50,artist_fav_q25, row['artist_favorites']), axis = 1)

  track_int_q75 = data_frame['track_interest'].quantile(0.75)
  track_int_q50 = data_frame['track_interest'].quantile(0.50)
  track_int_q25 = data_frame['track_interest'].quantile(0.25)
  data_frame['track_interest_level'] = data_frame.apply(lambda row: get_level(track_int_q75,track_int_q50,track_int_q25, row['track_interest']), axis = 1)

  track_lis_q75 = data_frame['track_listens'].quantile(0.75)
  track_lis_q50 = data_frame['track_listens'].quantile(0.50)
  track_lis_q25 = data_frame['track_listens'].quantile(0.25)
  data_frame['track_listens_level'] = data_frame.apply(lambda row: get_level(track_lis_q75,track_lis_q50,track_lis_q25, row['track_listens']), axis = 1)


  track_fav_q75 = data_frame['track_favorites'].quantile(0.75)
  track_fav_q50 = data_frame['track_favorites'].quantile(0.50)
  track_fav_q25 = data_frame['track_favorites'].quantile(0.25)
  data_frame['track_favorites_level'] = data_frame.apply(lambda row: get_level(track_fav_q75,track_fav_q50,track_fav_q25, row['track_favorites']), axis = 1)


  #normalziation 
  #data_frame['album_favorites_level'] = MinMaxScaler().fit_transform(np.array(data_frame['album_favorites_level']).reshape(-1,1))
  #data_frame['artist_favorites_level'] = MinMaxScaler().fit_transform(np.array(data_frame['artist_favorites_level']).reshape(-1,1))
  #data_frame['track_interest_level'] = MinMaxScaler().fit_transform(np.array(data_frame['track_interest_level']).reshape(-1,1))
  #data_frame['track_listens_level'] = MinMaxScaler().fit_transform(np.array(data_frame['track_listens_level']).reshape(-1,1))
  #data_frame['track_favorites_level'] = MinMaxScaler().fit_transform(np.array(data_frame['track_favorites_level']).reshape(-1,1))
  return data_frame

In [ ]:
convert_to_level(data_frame)
print(data_frame)

       album_favorites           album_title  artist_favorites artist_name  \
0                    4  AWOL - A Way Of Life                 9        AWOL   
1                    4  AWOL - A Way Of Life                 9        AWOL   
2                    4  AWOL - A Way Of Life                 9        AWOL   
3                    4     Constant Hitmaker                74   Kurt Vile   
4                    2                 Niris                10  Nicky Cook   
...                ...                   ...               ...         ...   
10082                2                   Eve                10     Shearer   
10083                2                   Eve                10     Shearer   
10084                2                   Eve                10     Shearer   
10085                2                   Eve                10     Shearer   
10086                2                   Eve                10     Shearer   

       track_favorites track_genre_top   track_genres   track_g

In [ ]:
def get_average(d):
  for k in d:
    d[k] = sum(d[k])/len(d[k])
  return d

In [ ]:
albums = {}
artists = {}
genres = {}
sond_id = 0

for index, row in data_frame.iterrows():
    # define albums
    album = str(row['album_title'])
    if not album in albums:
        albums[album] = [row['track_favorites_level']]
    else:
        albums[album].append(row['track_favorites_level'])
    # define artists
    artist = str(row['artist_name'])
    if not artist in artists:
        artists[artist] = [row['track_favorites_level']]
    else:
        artists[artist].append(row['track_favorites_level'])
    # define genres
    genre = str(row['track_genre_top'])
    if not genre in genres:
        genres[genre] = [row['track_favorites_level']]
    else:
        genres[genre].append(row['track_favorites_level'])
    sond_id+=1


dataset_size = sond_id
print("sond_id:", sond_id)
print("max index: ", index)
print("len(data_frame): ", len(data_frame))
albums = get_average(albums)
artists = get_average(artists)
genres = get_average(genres)

data_x = torch.zeros([dataset_size, 3])
data_y = torch.zeros([dataset_size], dtype=torch.int64)

song_id = 0
level = torch.zeros([4])
for index, row in data_frame.iterrows():
    data_x[song_id, 0] = albums[str(row['album_title'])]
    data_x[song_id, 1] = artists[str(row['artist_name'])]
    data_x[song_id, 2] = genres[str(row['track_genre_top'])]
    data_y[song_id] = row['track_favorites_level']
    level[row['track_favorites_level']] += 1
    song_id+=1

print(level)

sond_id: 9783
max index:  10086
len(data_frame):  9783
tensor([3866., 2192., 2004., 1721.])


In [ ]:
def shuffle(data_x, data_y):
  r=torch.randperm(data_x.shape[0])
  return data_x[r], data_y[r]

In [ ]:
data_x, data_y = shuffle(data_x, data_y)
train_size = int(0.8*data_x.shape[0])
train_x = data_x[:train_size]
train_y = data_y[:train_size]
test_x = data_x[train_size:]
test_y = data_y[train_size:]

# Model

In [ ]:
class LinearClassifier(nn.Module):
    def __init__(self):
        super(LinearClassifier, self).__init__()
        self.layer1 = nn.Linear(3, 4)

    def forward(self, x):
        out = self.layer1(x)
        return F.softmax(out)

class LinearClassifier2(nn.Module):
    def __init__(self):
        super(LinearClassifier2, self).__init__()
        self.layer1 = nn.Linear(3, 4)
        self.layer2 = nn.Linear(4,4)

    def forward(self, x):
        out = F.relu(self.layer1(x))
        out = self.layer2(out)

        return F.softmax(out)

In [ ]:
def test(model, batch_size, data_x, data_y):
  model.eval()
  f1_micro = F1Score(num_classes=4)
  f1_macro = F1Score(num_classes=4, average='macro')
  X = data_x
  y = data_y
  output = model(X)
  _, pred = torch.max(output, 1)
  num_correct = np.count_nonzero(pred.eq(y.data.view_as(pred)))
  num_samples = y.size(dim=0)

  accuracy = num_correct / num_samples
  return accuracy, f1_micro(pred, y), f1_macro(pred, y)

def train(model, lr, momentum, num_epochs, batch_size, data_x, data_y, test_x, test_y, displayinfo=True):
  opt = optim.SGD(model.parameters(), lr=lr,
                      momentum=momentum)
  
  criterion = nn.CrossEntropyLoss()
  dataset_size = data_x.shape[0]

  for epoch in range(1, num_epochs + 1):
    train_loss = 0
    model.train()
    num_correct = 0
    num_samples = 0
    for i in range(dataset_size//batch_size+1):
      start = i*batch_size
      end = min(start+batch_size, dataset_size)
      X = data_x[start:end]
      y = data_y[start:end]
      opt.zero_grad()
      output = model(X)
      loss = criterion(output,y)
      loss.backward()
      opt.step()
      train_loss += loss.item() * X.size(0)
      _, pred = torch.max(output, 1)
      num_correct += np.count_nonzero(pred.eq(y.data.view_as(pred)))
      num_samples += y.size(dim=0)

    train_accuracy = num_correct/num_samples
    test_accuracy, f1score_micro, f1score_macro = test(model, batch_size, test_x, test_y)
    if displayinfo:
      print("loss at epoch ", epoch, " : ", train_loss)
      print("Training accuracy at epoch ", epoch, " : ", train_accuracy)
      print(f"Test accuracy at epoch {epoch}: {test_accuracy:.4f}")
      print(f"Test f1score_micro at epoch {epoch}: {f1score_micro:.4f}")
      print(f"Test f1score_macro at epoch {epoch}: {f1score_macro:.4f}")
    if epoch == num_epochs:
      return train_accuracy, test_accuracy, f1score_micro, f1score_macro

In [ ]:
num_epochs = 30
lr = 0.5
momentum = 0.9
batch_size = 256

model = LinearClassifier()
train_accuracy, test_accuracy, f1score_micro, f1score_macro = train(model, lr, momentum, num_epochs, batch_size, train_x, train_y, test_x, test_y)

print(f"train_accuracy: {train_accuracy:.4f}")
print(f"test_accuracy: {test_accuracy:.4f}")
print(f"f1score_micro: {f1score_micro:.4f}")
print(f"f1score_macro: {f1score_macro:.4f}")

loss at epoch  1  :  9739.823960781097
Training accuracy at epoch  1  :  0.4723996933299259
Test accuracy at epoch 1: 0.6045
Test f1score_micro at epoch 1: 0.6045
Test f1score_macro at epoch 1: 0.4784
loss at epoch  2  :  8905.532413959503
Training accuracy at epoch  2  :  0.6056733963710708
Test accuracy at epoch 2: 0.6157
Test f1score_micro at epoch 2: 0.6157
Test f1score_macro at epoch 2: 0.4930
loss at epoch  3  :  8813.194992542267
Training accuracy at epoch  3  :  0.6242013800153335
Test accuracy at epoch 3: 0.6306
Test f1score_micro at epoch 3: 0.6306
Test f1score_macro at epoch 3: 0.5129
loss at epoch  4  :  8743.307918787003
Training accuracy at epoch  4  :  0.6337848198313315
Test accuracy at epoch 4: 0.6290
Test f1score_micro at epoch 4: 0.6290
Test f1score_macro at epoch 4: 0.5112
loss at epoch  5  :  8625.826308012009
Training accuracy at epoch  5  :  0.6340403782264248
Test accuracy at epoch 5: 0.6239
Test f1score_micro at epoch 5: 0.6239
Test f1score_macro at epoch 5: 0.

In [ ]:
num_epochs = 30
lr_list = [0.001, 0.01, 0.1, 0.2, 0.5, 1, 5]
momentum_list = [0, 0.1, 0.2, 0.5, 0.8, 0.9, 1]
batch_size = 256

train_accuracies = torch.zeros([len(lr_list), len(momentum_list)])
test_accuracies = torch.zeros([len(lr_list), len(momentum_list)])
test_f1score_micro = torch.zeros([len(lr_list), len(momentum_list)])
test_f1score_macro = torch.zeros([len(lr_list), len(momentum_list)])

for i in range(len(lr_list)):
  lr = lr_list[i]
  for j in range(len(momentum_list)):
    momentum = momentum_list[j]
    model = LinearClassifier()
    train_accuracy, test_accuracy, f1score_micro, f1score_macro = train(model, lr, momentum, num_epochs, batch_size, train_x, train_y, test_x, test_y, displayinfo=False)
    train_accuracies[i,j] = train_accuracy
    test_accuracies[i,j] = test_accuracy
    test_f1score_micro[i,j] = f1score_micro
    test_f1score_macro[i,j] = f1score_macro
    print(f"lr: {lr}, momentum: {momentum}, train_accuracy: {train_accuracy:.4f}, test_accuracy: {test_accuracy:.4f}, f1score_micro: {f1score_micro:.4f}, f1score_macro: {f1score_macro:.4f}")

print("train_accuracies: ", train_accuracies)
print("test_accuracies: ", test_accuracies)
print("test_f1score_micro: ", test_f1score_micro)
print("test_f1score_macro: ", test_f1score_macro)

lr: 0.001, momentum: 0, train_accuracy: 0.2296, test_accuracy: 0.2264, f1score_micro: 0.2264, f1score_macro: 0.1022
lr: 0.001, momentum: 0.1, train_accuracy: 0.5891, test_accuracy: 0.5846, f1score_micro: 0.5846, f1score_macro: 0.4616
lr: 0.001, momentum: 0.2, train_accuracy: 0.1797, test_accuracy: 0.1620, f1score_micro: 0.1620, f1score_macro: 0.0697
lr: 0.001, momentum: 0.5, train_accuracy: 0.2029, test_accuracy: 0.1998, f1score_micro: 0.1998, f1score_macro: 0.1126
lr: 0.001, momentum: 0.8, train_accuracy: 0.5098, test_accuracy: 0.5294, f1score_micro: 0.5294, f1score_macro: 0.3164
lr: 0.001, momentum: 0.9, train_accuracy: 0.5382, test_accuracy: 0.5539, f1score_micro: 0.5539, f1score_macro: 0.3295
lr: 0.001, momentum: 1, train_accuracy: 0.5521, test_accuracy: 0.5432, f1score_micro: 0.5432, f1score_macro: 0.3299
lr: 0.01, momentum: 0, train_accuracy: 0.5210, test_accuracy: 0.5156, f1score_micro: 0.5156, f1score_macro: 0.3206
lr: 0.01, momentum: 0.1, train_accuracy: 0.5488, test_accuracy: